In [2]:
import json
proxy_config = None
with open('proxy_config.json') as json_data:
    proxy_config = json.load(json_data)

proxy_port_range=range(proxy_config["proxy_port_start"], proxy_config["proxy_port_end"])
proxies=[f'https://{proxy_config["proxy_user"]}:{proxy_config["proxy_pwd"]}@{proxy_config["proxy_url"]}:{port}' for port in proxy_port_range]

In [10]:
%cd jobspy
#I have to use this until my fix gets incorporated into the package

/home/ekaterina/dev/JobsAnalyzer/jobspy


In [12]:
from jobspy import scrape_jobs
general_scraper_config = {
    "search_term": "machine learning",
    "google_search_term": "machine learning ",
    "location": "European Union",
    "results_wanted": 200,
    "hours_old": 720,
    "is_remote": True,
    "proxies":proxies,
}

scraper_configs = dict()

scraper_configs["indeed"] = general_scraper_config.copy()
scraper_configs["indeed"]["site_name"] = ["indeed"]
scraper_configs["indeed"]["country_indeed"] = "Germany"

scraper_configs["glassdoor"] = general_scraper_config.copy()
scraper_configs["glassdoor"]["site_name"] = ["glassdoor"]
scraper_configs["glassdoor"]["location"] = "Germany"
scraper_configs["glassdoor"]["is_remote"] = False # glassdoor mostly scrapes US jobs if remote is set to true
scraper_configs["glassdoor"]["proxies"] = None

scraper_configs["google"] = general_scraper_config.copy()
scraper_configs["google"]["site_name"] = ["google"]

scraper_configs["linkedin"] = general_scraper_config.copy()
scraper_configs["linkedin"]["site_name"] = ["linkedin"]
scraper_configs["linkedin"]["linkedin_fetch_description"] = True # gets more info such as description, direct job url (slower)

In [13]:
jobs_per_site = dict()
for site, s_config in scraper_configs.items():
    jobs_per_site[site] =  scrape_jobs(**s_config)

2025-03-03 11:08:06,147 - INFO - JobSpy:Linkedin - finished scraping


In [14]:
import pandas as pd
import csv
jobs = pd.concat([job_list for site, job_list in jobs_per_site.items()], ignore_index=False)

/tmp/ipykernel_1303221/2529843683.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jobs = pd.concat([job_list for site, job_list in jobs_per_site.items()], ignore_index=False)


In [17]:
from langdetect import detect
language = []
for desc in jobs['description']:
    try: 
        language.append(detect(desc))
    except:
        language.append(None)

In [18]:
jobs["language"] = language
jobs.to_csv("jobs.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=True) 

In [60]:
# Count the number of jobs that mention a location in Germany
germany_jobs_count = jobs[jobs['location'].str.contains("Germany", na=False)].shape[0]
total_jobs_count = jobs.shape[0]

# Percentage of jobs in Germany
germany_jobs_count, total_jobs_count, (germany_jobs_count / total_jobs_count) * 100


(14, 482, 2.904564315352697)

In [61]:
jobs

,id,site,job_url,job_url_direct,title,company,location,date_posted,job_type,salary_source,...,description,company_industry,company_url,company_logo,company_url_direct,company_addresses,company_num_employees,company_revenue,company_description,language
0,gd-1009654460000,glassdoor,https://www.glassdoor.com/job-listing/j?jl=100...,None,"Full-stack Developer, Trilogy (Remote) - $60,0...",Crossover,NaN,2025-02-28,None,NaN,...,Do you have a knack for software engineering a...,None,https://www.glassdoor.com/Overview/W-EI_IE1041...,https://media.glassdoor.com/sql/1041838/crosso...,None,None,None,None,None,en
1,gd-1009653539443,glassdoor,https://www.glassdoor.com/job-listing/j?jl=100...,None,DIRECTOR AI & MACHINE LEARNING,PUMA,Herzogenaurach,2025-02-28,None,NaN,...,SPEED \& SPIRIT is what we look for in our can...,None,https://www.glassdoor.com/Overview/W-EI_IE1047...,https://media.glassdoor.com/sql/10474/puma-squ...,None,None,None,None,None,en
2,gd-1009653653116,glassdoor,https://www.glassdoor.com/job-listing/j?jl=100...,None,"Applied Scientist, Amazon Music Machine Learni...",Amazon Development Center Germany GmbH,Berlin,2025-02-28,None,NaN,...,"\- PhD, or a Master's degree and experience in...",None,https://www.glassdoor.com/Overview/W-EI_IE6036...,https://media.glassdoor.com/sql/6036/amazon-sq...,None,None,None,None,None,en
3,gd-1009422458687,glassdoor,https://www.glassdoor.com/job-listing/j?jl=100...,None,"Software Development Engineer, Trilogy (Remote...",Crossover,Berlin,2025-02-28,None,NaN,...,Ready to revolutionize the telecom industry wi...,None,https://www.glassdoor.com/Overview/W-EI_IE1041...,https://media.glassdoor.com/sql/1041838/crosso...,None,None,None,None,None,en
4,gd-1009653361385,glassdoor,https://www.glassdoor.com/job-listing/j?jl=100...,None,Senior Data Engineer (m/w/d),Thermengruppe Josef Wund,Sinsheim,2025-02-28,None,NaN,...,#### **Das sind WIR...**\n\n\nWir bieten dir e...,None,https://www.glassdoor.com/Overview/W-EI_IE7528...,https://media.glassdoor.com/sql/7528869/wund-h...,None,None,None,None,None,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,li-4169091856,linkedin,https://www.linkedin.com/jobs/view/4169091856,NaN,Manager Data Science & AI,Avance Consulting,,NaN,contract,None,...,* **Independent lateral management of cross\-f...,IT Services and IT Consulting,https://uk.linkedin.com/company/avance-services,https://media.licdn.com/dms/image/v2/D560BAQGM...,None,None,None,None,None,en
478,li-4169068056,linkedin,https://www.linkedin.com/jobs/view/4169068056,NaN,Data Scientist (Full remote) - IT,ALTEN Italia,,NaN,fulltime,None,...,**Ruolo:** \n Data Scientist\n \n\n\n\n \n\n\...,IT Services and IT Consulting,https://it.linkedin.com/company/altenitalia,https://media.licdn.com/dms/image/v2/C4D0BAQHw...,None,None,None,None,None,it
479,li-4166297549,linkedin,https://www.linkedin.com/jobs/view/4166297549,NaN,"Python Developer (Freelance, 100% Remote)",DevologyX,,NaN,fulltime,None,...,"ROLE OVERVIEW\n \n\n\n\n 12 Month Contract, Fu...",Staffing and Recruiting,https://ee.linkedin.com/company/devologyx,https://media.licdn.com/dms/image/v2/D4E0BAQFr...,None,None,None,None,None,en
480,li-4169054756,linkedin,https://www.linkedin.com/jobs/view/4169054756,NaN,Business Analytics Training - Internship,Oeson Learning™,,NaN,other,None,...,**About the Company:** \n\n\n\n\n Oeson is a l...,IT Services and IT Consulting,https://in.linkedin.com/company/oesonlearning,https://media.licdn.com/dms/image/v2/C4D0BAQG6...,None,None,None,None,None,en


In [35]:
jobs.columns

Index(['id', 'site', 'job_url', 'job_url_direct', 'title', 'company',
       'location', 'date_posted', 'job_type', 'salary_source', 'interval',
       'min_amount', 'max_amount', 'currency', 'is_remote', 'job_level',
       'job_function', 'listing_type', 'emails', 'description',
       'company_industry', 'company_url', 'company_logo', 'company_url_direct',
       'company_addresses', 'company_num_employees', 'company_revenue',
       'company_description', 'language'],
      dtype='object')